In [1]:
#Import the Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime
import requests
# Import the API key.
from config import weather_api_key

In [2]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [3]:
#Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [4]:
coordinates = list(lat_lngs)

In [5]:
#Create a list for holding the cities
cities = []
#Identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#print the city count to confirm sufficient count
len(cities)

758

In [6]:
#Create an empty list to hold the the weather data
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval.       ")
print("--------------------------------")

#Create counters.
record_count = 1
set_count = 1

#Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Log the URL, record, and set unbers and the city
    #print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count +=1
    
    #Run and API request for each of the cities.
    try:
        #Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        #Parse the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        #Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "City Description": city_description,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        #print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval.       
--------------------------------
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
len(city_data)

697

In [13]:
#Convert the array of dictionaries to a Pandas dataframe
city_data_df = pd.DataFrame(city_data)
#city_data_df.head(10)

In [14]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed","City Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,City Description
0,Ribeira Grande,PT,2021-04-23 17:49:26,38.5167,-28.7000,60.80,67,20,36.82,few clouds
1,Hilo,US,2021-04-23 17:43:51,19.7297,-155.0900,71.60,73,75,3.44,light rain
2,Ndende,GA,2021-04-23 17:53:27,-2.4008,11.3581,77.34,86,61,4.50,light rain
3,Bethel,US,2021-04-23 17:49:41,41.3712,-73.4140,57.00,26,1,17.27,clear sky
4,Busselton,AU,2021-04-23 17:43:43,-33.6500,115.3333,62.01,77,66,1.99,broken clouds
5,Saint-Augustin,CA,2021-04-23 17:45:50,51.2260,-58.6502,35.69,68,100,14.83,overcast clouds
6,Kapaa,US,2021-04-23 17:44:12,22.0752,-159.3190,73.40,78,40,12.66,scattered clouds
7,Butaritari,KI,2021-04-23 17:45:11,3.0707,172.7902,81.95,76,95,11.01,overcast clouds
8,Atuona,PF,2021-04-23 17:41:48,-9.8000,-139.0333,80.29,80,26,21.54,scattered clouds
9,Surt,LY,2021-04-23 17:53:28,31.2089,16.5887,81.93,34,64,8.12,broken clouds


In [16]:
# Create the output file (CSV)
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the dataframe to a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")